<a href="https://colab.research.google.com/github/tjturnage/radar/blob/main/Download_AWS_WSR_88D_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **This script allows the user to download selected WSR-88D Archive Level 2 files from Amazon Web Services**

Retrieved files will be placed into a zip file shown to left that can then be downloaded.

Authors:
* Eric Lenning -- NWS Chicago -- @eric.lenning@noaa.gov
* TJ Turnage -- NWS Grand Rapids -- @thomas.turnage@noaa.gov


In [ ]:
# @title Install the necessary modules
import warnings
warnings.filterwarnings('ignore')
!pip install s3fs &> /dev/null
import s3fs
import sys
import os
import math
from datetime import timedelta
from datetime import datetime
import numpy as np

class NexradLevel2():

    def __init__(self, site, start_datetime, end_datetime):
        self.site = site
        self.start_datetime = start_datetime
        self.end_datetime = end_datetime

    def daterange(self):
        """ Yields list of dates within specified time range. """

        span = self.end_datetime - self.start_datetime

        day_span = math.ceil(float(span.total_seconds()) / (3600*24))

        if self.start_datetime.hour > self.end_datetime.hour:
            day_span += 1

        for n in range(day_span):
            yield self.start_datetime + timedelta(n)

    def filelist(self):
        """ List of files for site within specified time range. """

        fs = s3fs.S3FileSystem(anon=True)
        fs.ls('s3://noaa-nexrad-level2/')

        radarfiles = []

        for single_date in self.daterange():

            YYYY = single_date.year  # strftime("%Y")
            mm = single_date.month  # strftime("%m")
            dd = single_date.day  # strftime("%d")

            # sample bucket dir : 'noaa-nexrad-level2/2018/07/19/KDMX/'
            # broken into two substrings to stay <79 columns
            bucket_str1 = f'noaa-nexrad-level2/{YYYY:.0f}/'
            bucket_str2 = f'{mm:02.0f}/{dd:02.0f}/{self.site}/'
            bucket_dir_str = bucket_str1 + bucket_str2
            # list available files in bucket
            # sample filename :  KDMX20180719_221153_V06
            files = np.array(fs.ls(bucket_dir_str))

            for f in range(0, len(files)):

                filename = files[f].split('/')[-1]  # extracts fname after /

                if 'MDM' not in filename:
                    file_date = filename[4:19]

                    try:
                        file_datetime = datetime.strptime(file_date,
                                                          '%Y%m%d_%H%M%S')

                        if (file_datetime >= self.start_datetime
                           and file_datetime <= self.end_datetime):

                            print(files[f])

                            radarfiles.append(files[f])

                            info = fs.info(files[f])

                            #print("size ", info['size'])

                            #print(info)

                    except Exception:
                        print("Unexpected error:", sys.exc_info()[0])

        return radarfiles

    def download(self, filelist):
        """Download level 2 radar files from AWS.

        Files will be named according to the format on AWS but assumes nothing
        about the destination folder.  self is because different users may
        have different requirements for where data should be stored.

        Args:
            filelist: A list of files in the AWS NEXRAD inventory.
            raw_data_dir: Full pathname of download destination.

        Returns:
            The return value. True for success, False otherwise.
        """

        #os.makedirs(raw_data_dir, exist_ok=True)

        fs = s3fs.S3FileSystem(anon=True)

        # https://noaa-nexrad-level2.s3.amazonaws.com/
        #           2015/04/10/KLOT/KLOT20150410_235635_V06.gz
        # sample filename :  KDMX20180719_221153_V06

        # downloaded = []

        download_count = 0

        # sample source filepath
        # 'noaa-nexrad-level2/2018/07/19/KDMX/KDMX20180719_221153_V06'
        for f in range(0, len(filelist)):
            try:
                print('getting... ' + str(filelist[f]))
                dst_filepath = os.path.join('radarfiles', filelist[f].split('/')[-1])

                #print('...to ', dst_filepath)

                info = fs.info(filelist[f])

                remote_filesize = info['size']

                try:
                    stat = os.stat(dst_filepath)
                    #print(stat)
                    local_filesize = stat.st_size

                    print('remote and local filesize: ',
                          remote_filesize, local_filesize)

                    if local_filesize < remote_filesize:
                        raise Exception('File exists but is too small.')

                    print('Already downloaded.')
                    download_count += 1
                except Exception:
                    fs.get(filelist[f], dst_filepath)  # download to dest dir
                    print('  Download complete!')
                    download_count += 1
                    # downloaded.append(filelist[f])
            except Exception:
                pass

        # Tell caller if you got all the files it wanted.
        if download_count == len(filelist):
            return True
        else:
            return False


In [ ]:
# @markdown <br><font color="blue"><h2>Select the radar to download data for</h2></font>
radarId = "KGRR" # @param {type:"string"}

# @markdown <br><font color="blue"><h2>Enter the start time (in UTC) for the radar data download</h2></font>
start_ymd = "2024-02-28" # @param {type:"date"}
start_hour = "03" # @param ["00","01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17,","18","19","20","21","22","23"]
start_minute = "00" # @param ["00", "05", "10", "15", "20", "25", "30", "35", "40", "45", "50", "55"]
start_time_str = f"{start_ymd} {start_hour} {start_minute}"
start_date = datetime.strptime(start_time_str, "%Y-%m-%d %H %M")


# @markdown <br><font color="blue"><h2>Enter the end time (in UTC) for the radar data download</h2></font>
end_ymd = "2024-02-28" # @param {type:"date"}
end_hour = "06" # @param ["00","01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17,","18","19","20","21","22","23"]
end_minute = "45" # @param ["00", "05", "10", "15", "20", "25", "30", "35", "40", "45", "50", "55"]

end_time_str = f"{end_ymd} {end_hour} {end_minute}"
end_date = datetime.strptime(end_time_str, "%Y-%m-%d %H %M")

# datetime(year, month, day, hour, minute)
# example: datetime(2023, 6, 16, 3, 0)

print(f"start time: {start_date}")
print(f"  end time: {end_date}")


<h3>Running the code below will download files to a "radarfiles" directory that is then made into a "radarfiles.zip" file that can be downloaded. If you don't see these appear right away, click the folder refresh icon at the top of the Files window.</h3>

<h4><i>- Right-click over "radarfiles.zip" and select Download <br>- There will be a delay as the zip file preps for download<br>- You can monitor progress with the growing circle on the right</i></h4>

In [ ]:
# @title
example = NexradLevel2(radarId, start_date, end_date)
example.download(example.filelist())

!zip -r /content/radarfiles.zip /content/radarfiles